In [19]:
from langchain.chat_models import ChatOllama
from langchain.schema import HumanMessage, AIMessage, SystemMessage, BaseOutputParser
from langchain.prompts import PromptTemplate, ChatPromptTemplate

chat = ChatOllama(model="mistral:latest")

class CommaOutputParser(BaseOutputParser):
    def parse(self, text):
        itmes = text.strip().split(",")
        return list(map(str.strip, itmes))
    
p = CommaOutputParser()

p.parse("Hello, how, are, you")


['Hello', 'how', 'are', 'you']

In [1]:
from langchain.chat_models import ChatOllama
from langchain.prompts import PromptTemplate, ChatPromptTemplate
from langchain.callbacks import StreamingStdOutCallbackHandler
from langchain.memory import ConversationBufferMemory

chat = ChatOllama(model="mistral:latest", streaming=True, callbacks=[StreamingStdOutCallbackHandler()])

memory = ConversationBufferMemory(return_messages=True)

memory.save_context({"input": "Hi!"}, {"output": "How are you?"})

memory.load_memory_variables({})


{'history': [HumanMessage(content='Hi!'), AIMessage(content='How are you?')]}

In [ ]:
from langchain.memory import ConversationSummaryBufferMemory
from langchain.chat_models import ChatOpenAI
from langchain.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain.schema.runnable import RunnablePassthrough


llm = ChatOpenAI(temperature=0.1)

memory = ConversationSummaryBufferMemory(
    llm=llm,
    max_token_limit=120,
    memory_key="chat_history",
    return_messages=True,
)

prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "You are a helpful AI talking to a human"),
        MessagesPlaceholder(variable_name="chat_history"),
        ("human", "{question}"),
    ]
)

def load_memory(input):
    return memory.load_memory_variables({})["chat_history"]

chain = RunnablePassthrough.assign(chat_history = load_memory) | prompt | llm


def invoke_chain(question):
    result = chain.invoke({"question": question})
    memory.save_context({"input": question}, {"output": result.content})

invoke_chain("My name is Jeon")
invoke_chain("What is my name?")

In [12]:
"""
messages = [
    SystemMessage(content="You are geography export."),
    AIMessage(content="There are many ways to measure distance"),
    HumanMessage(content="What is the distance between Mexico and Thailand?")
]

chat.predict_messages(messages)       
"""



"""
template = ChatPromptTemplate.from_messages([
    ("system", "You are a geography expert."),
    ("ai", "I reply to your answers with numbers"),
    ("human", "What is the distance between {country_a} and {country_b}.")
])


prompt = template.format_messages(
    country_a = "South Korea",
    country_b = "Japan",
)

chat.predict_messages(prompt)
"""

'\nmessages = [\n    SystemMessage(content="You are geography export."),\n    AIMessage(content="There are many ways to measure distance"),\n    HumanMessage(content="What is the distance between Mexico and Thailand?")\n]\n\nchat.predict_messages(messages)       \n'

In [ ]:
from langchain.chat_models import ChatOllama
from langchain.schema import HumanMessage, AIMessage, SystemMessage, BaseOutputParser
from langchain.prompts import PromptTemplate, ChatPromptTemplate
from langchain.callbacks import StreamingStdOutCallbackHandler

chat = ChatOllama(model="mistral:latest", streaming=True, callbacks=[StreamingStdOutCallbackHandler()])

chef_prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a world-class international chef. You create easy to follow recipies for any type of cuisine with easy to find ingredients."),
    ("human", "I want to cook {cuisine} food."),
])

chef_chain = chef_prompt | chat


veg_chef_prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a vegetarian chef specialized on making traditional recipies vegetarian. You find alternative ingredients and explain their preparation. You don't radically modify the recipe. If there is no alternative for a food just say you don't know how to replace it."),
    ("human", "{recipe}"),
])


veg_chain = veg_chef_prompt | chat


final_chain = {"recipe" : chef_chain} | veg_chain

final_chain.invoke({
    "cuisine" : "indian"
})


In [ ]:
from langchain.chat_models import ChatOllama
from langchain.callbacks import StreamingStdOutCallbackHandler
from langchain.document_loaders import UnstructuredFileLoader
from langchain.text_splitter import CharacterTextSplitter

splitter = CharacterTextSplitter.from_tiktoken_encoder(
    separator="\n",
    chunk_size = 600,
    chunk_overlap = 100,
)

loader = UnstructuredFileLoader("./files/chapter_one.docx")

loader.load_and_split(text_splitter=splitter)



In [1]:
from langchain.chat_models import ChatOllama
from langchain.schema import HumanMessage, AIMessage, SystemMessage, BaseOutputParser
from langchain.prompts import PromptTemplate, ChatPromptTemplate
from langchain.callbacks import StreamingStdOutCallbackHandler
from langchain.prompts.few_shot import FewShotPromptTemplate

chat = ChatOllama(model="mistral:latest", streaming=True, callbacks=[StreamingStdOutCallbackHandler()])


examples = [
    {
        "question" : "What do you know about France?",
        "answer": """
        Here is what I know:
        Capital: Paris
        Language: France
        Food: Wine and Cheese
        Currency: Euro
        """,
    },
    {
        "question" : "What do you know about Italy?",
        "answer": """
        Here is what I know:
        Capital: Rome
        Language: Italian
        Food: Pizza and Pasta
        Currency: Euro
        """, 
    },
    {
        "question" : "What do you know about Greece?",
        "answer": """
        Here is what I know:
        Capital: Athens
        Language: Greek
        Food: Souvlaki and Feta Cheese
        Currency: Euro
        """, 
    },
]



example_prompt = PromptTemplate.from_template("Human: {question}\nAI: {answer}")


prompt = FewShotPromptTemplate(
    example_prompt = example_prompt,
    examples = examples,
    suffix= "Human: What do you know about {country}?",
    input_variables=["country"]
)

chain = prompt | chat

chain.invoke({
    "country": "Germany"
})




 Here is what I know:
Capital: Berlin
Language: German
Food: Sausages, Pretzels, and Beer
Currency: Euro. However, it's important to note that Germany was part of the Deutsche Mark zone before it adopted the Euro in 2002.

AIMessage(content=" Here is what I know:\nCapital: Berlin\nLanguage: German\nFood: Sausages, Pretzels, and Beer\nCurrency: Euro. However, it's important to note that Germany was part of the Deutsche Mark zone before it adopted the Euro in 2002.")

In [18]:
from langchain.chat_models import ChatOllama
from langchain.prompts import ChatPromptTemplate
from langchain.callbacks import StreamingStdOutCallbackHandler
from langchain.prompts.few_shot import FewShotPromptTemplate, FewShotChatMessagePromptTemplate
from langchain.prompts.example_selector import LengthBasedExampleSelector


chat = ChatOllama(model="mistral:latest", streaming=True, callbacks=[StreamingStdOutCallbackHandler()])

examples = [
    {
        "country" : "France",
        "answer": """
        Here is what I know:
        Capital: Paris
        Language: France
        Food: Wine and Cheese
        Currency: Euro
        """,
    },
    {
        "country" : "Italy",
        "answer": """
        Here is what I know:
        Capital: Rome
        Language: Italian
        Food: Pizza and Pasta
        Currency: Euro
        """, 
    },
    {
        "country" : "Greece",
        "answer": """
        Here is what I know:
        Capital: Athens
        Language: Greek
        Food: Souvlaki and Feta Cheese
        Currency: Euro
        """, 
    },
]


example_prompt = ChatPromptTemplate.from_messages([
    ("human", "What do you know about {country}?"),
    ("ai", "{answer}")
])


chat_prompt = FewShotChatMessagePromptTemplate(
    example_prompt = example_prompt,
    examples = examples,
)

final_prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a geography expert and you give short answer for easy understanding"),
    chat_prompt,
    ("human", "What do you know about {country}?")
])

chain = final_prompt | chat

chain.invoke({
    "country": "South Korea"
})



 Here is what I know:
Capital: Seoul
Language: Korean (both North and South have their own versions)
Food: Kimchi and Bulgogi
Currency: South Korean won
South Korea is located in East Asia and is divided into two distinct regions, North Korea and South Korea. It has a rich history with ancient ruins, dynamic cities and a unique culture all its own.

AIMessage(content=' Here is what I know:\nCapital: Seoul\nLanguage: Korean (both North and South have their own versions)\nFood: Kimchi and Bulgogi\nCurrency: South Korean won\nSouth Korea is located in East Asia and is divided into two distinct regions, North Korea and South Korea. It has a rich history with ancient ruins, dynamic cities and a unique culture all its own.')

In [28]:
from langchain.chat_models import ChatOllama
from langchain.prompts import PromptTemplate, ChatPromptTemplate
from langchain.callbacks import StreamingStdOutCallbackHandler
from langchain.prompts.few_shot import FewShotPromptTemplate
from langchain.prompts.example_selector import LengthBasedExampleSelector
from langchain.prompts.example_selector.base import BaseExampleSelector
from langchain.prompts import example_selector



chat = ChatOllama(model="mistral:latest", streaming=True, callbacks=[StreamingStdOutCallbackHandler()])


examples = [
    {
        "question" : "What do you know about France?",
        "answer": """
        Here is what I know:
        Capital: Paris
        Language: France
        Food: Wine and Cheese
        Currency: Euro
        """,
    },
    {
        "question" : "What do you know about Italy?",
        "answer": """
        Here is what I know:
        Capital: Rome
        Language: Italian
        Food: Pizza and Pasta
        Currency: Euro
        """, 
    },
    {
        "question" : "What do you know about Greece?",
        "answer": """
        Here is what I know:
        Capital: Athens
        Language: Greek
        Food: Souvlaki and Feta Cheese
        Currency: Euro
        """, 
    },
]


class RandomExampleSelector(BaseExampleSelector):
    def __init__(self, examples):
        self.examples = examples

    def add_example(self, example):
        self.examples.append(example)

    def select_examples(self, input_variables):
        from random import choice

        return [choice(self.examples)]


example_prompt = PromptTemplate.from_template("Human: {question}\nAI:{answer}")

example_selector = RandomExampleSelector(
    examples=examples,
)

prompt = FewShotPromptTemplate(
    example_prompt=example_prompt,
    example_selector=example_selector,
    suffix="Human: What do you know about {country}?",
    input_variables=["country"],
)

prompt.format(country="Brazil")

'Human: What do you know about France?\nAI:\n        Here is what I know:\n        Capital: Paris\n        Language: France\n        Food: Wine and Cheese\n        Currency: Euro\n        \n\nHuman: What do you know about Brazil?'

In [10]:
from langchain.chat_models import ChatOllama
from langchain.prompts import PromptTemplate, ChatPromptTemplate
from langchain.callbacks import StreamingStdOutCallbackHandler
from langchain.prompts.pipeline import PipelinePromptTemplate

chat = ChatOllama(model="mistral:latest", streaming=True, callbacks=[StreamingStdOutCallbackHandler()])


intro = PromptTemplate.from_template(
"""
You are a role playing assistant.
And you are impersonating a {character}
"""
)

example = PromptTemplate.from_template(
"""
This is an example of how you talk:

Human: {example_question}
You: {example_answer}
"""
)

start = PromptTemplate.from_template(
"""
Start now!

Human: {question}
You:
"""
)

final = PromptTemplate.from_template(
"""
{intro}

{example}

{start}
"""
)


prompts = [
    ("intro", intro),
    ("example", example),
    ("start", start)
]

full_prompt = PipelinePromptTemplate(final_prompt = final, pipeline_prompts=prompts)

"""
full_prompt.format(
    character = "Pirate",
    example_question = "What is your location?",
    example_answer = "Arrrg! That is a secret!",
    question = "What is your favorite food?"
)
"""

chain = full_prompt | chat

chain.invoke({
    "character" : "Pirate",
    "example_question" : "What is your location?",
    "example_answer" : "Arrrg! That is a secret!",
    "question" : "What is your favorite food?"
})


 Arrrrgh! Me hearty loves a good plate o' pirate stew, made with fresh fish, ripe fruits, and hardtack bread. Avast ye, landlubber!

AIMessage(content=" Arrrrgh! Me hearty loves a good plate o' pirate stew, made with fresh fish, ripe fruits, and hardtack bread. Avast ye, landlubber!")

In [14]:
from langchain.chat_models import ChatOllama
from langchain.prompts import PromptTemplate, ChatPromptTemplate
from langchain.callbacks import StreamingStdOutCallbackHandler
from langchain.globals import set_llm_cache, set_debug
from langchain.cache import InMemoryCache, SQLiteCache

set_llm_cache(SQLiteCache("cache.db"))
set_debug(True)

chat = ChatOllama(model="mistral:latest")

chat.predict("How do you make italian pasta?")





[llm/start] [1:llm:ChatOllama] Entering LLM run with input:
{
  "prompts": [
    "Human: How do you make italian pasta?"
  ]
}
[llm/end] [1:llm:ChatOllama] [262.72s] Exiting LLM run with output:
{
  "generations": [
    [
      {
        "text": " Making authentic Italian pasta from scratch involves several steps. Here's a simplified version of the process for making classic spaghetti or fettuccine using a pasta machine:\n\n**Ingredients:**\n- 200 g durum wheat semolina\n- 3 eggs (large)\n- Water, as needed\n- Salt, to taste\n\n**Instructions:**\n\n1. **Prepare the dough:** Make a well in the center of a large pile of semolina on a clean, wooden surface or countertop. Crack the eggs into the well. Use a fork to gently whisk the eggs and gradually incorporate the surrounding semolina, creating a thick paste.\n2. **Knead the dough:** Once all the semolina is incorporated, start kneading the dough using the heel of your hand, pushing it away from you and then folding it back over itself. 

" Making authentic Italian pasta from scratch involves several steps. Here's a simplified version of the process for making classic spaghetti or fettuccine using a pasta machine:\n\n**Ingredients:**\n- 200 g durum wheat semolina\n- 3 eggs (large)\n- Water, as needed\n- Salt, to taste\n\n**Instructions:**\n\n1. **Prepare the dough:** Make a well in the center of a large pile of semolina on a clean, wooden surface or countertop. Crack the eggs into the well. Use a fork to gently whisk the eggs and gradually incorporate the surrounding semolina, creating a thick paste.\n2. **Knead the dough:** Once all the semolina is incorporated, start kneading the dough using the heel of your hand, pushing it away from you and then folding it back over itself. Continue this process until the dough becomes elastic and smooth. This may take 10-15 minutes. Add a little water if necessary to keep the dough from becoming too dry.\n3. **Rest the dough:** Wrap the dough in plastic wrap and let it rest for at 

In [17]:
from langchain.chat_models import ChatOllama
from langchain.prompts import PromptTemplate, ChatPromptTemplate
from langchain.callbacks import StreamingStdOutCallbackHandler
from langchain.memory import ConversationBufferMemory

chat = ChatOllama(model="mistral:latest", streaming=True, callbacks=[StreamingStdOutCallbackHandler()])

memory = ConversationBufferMemory(return_messages=True)

memory.save_context({"input": "Hi!"}, {"output": "How are you?"})

memory.load_memory_variables({})

{'history': [HumanMessage(content='Hi!'), AIMessage(content='How are you?')]}

In [19]:
"""

from langchain.chat_models import ChatOllama
from langchain.prompts import PromptTemplate, ChatPromptTemplate
from langchain.callbacks import StreamingStdOutCallbackHandler
from langchain.memory import ConversationBufferWindowMemory

chat = ChatOllama(model="mistral:latest", streaming=True, callbacks=[StreamingStdOutCallbackHandler()])

memory = ConversationBufferWindowMemory(
    return_messages=True,
    k = 4
)


def add_message(input, output):
    memory.save_context({"input": input}, {"output": output})

add_message(1, 1)

"""


In [21]:
"""
add_message(2, 2)
add_message(3, 3)
add_message(4, 4)
add_message(5, 5)
memory.load_memory_variables({})
"""

{'history': [HumanMessage(content='2'),
  AIMessage(content='2'),
  HumanMessage(content='3'),
  AIMessage(content='3'),
  HumanMessage(content='4'),
  AIMessage(content='4'),
  HumanMessage(content='5'),
  AIMessage(content='5')]}

In [5]:
from langchain.chat_models import ChatOllama
from langchain.prompts import PromptTemplate, ChatPromptTemplate
from langchain.callbacks import StreamingStdOutCallbackHandler
from langchain.memory import ConversationSummaryBufferMemory

chat = ChatOllama(model="mistral:latest", streaming=True, callbacks=[StreamingStdOutCallbackHandler()])

memory = ConversationSummaryBufferMemory(llm=chat)

def add_message(input, output):
    memory.save_context({"input": input}, {"output": output})

def get_history():
    return memory.load_memory_variables({})

add_message("Hi I'm Jeon, I live in Tokyo", "Wow that is so cool!")






In [ ]:
from langchain.memory import ConversationSummaryMemory
from langchain.chat_models import ChatOpenAI

llm = ChatOpenAI(temperature=0.1)

memory = ConversationSummaryMemory(llm=llm)


def add_message(input, output):
    memory.save_context({"input": input}, {"output": output})


def get_history():
    return memory.load_memory_variables({})


add_message("Hi I'm Nicolas, I live in South Korea", "Wow that is so cool!")

add_message("South Kddorea is so pretty", "I wish I could go!!!")

get_history()

In [ ]:
from langchain.memory import ConversationSummaryBufferMemory
from langchain.chat_models import ChatOpenAI

llm = ChatOpenAI(temperature=0.1)

memory = ConversationSummaryBufferMemory(
    llm=llm,
    max_token_limit=150,
    return_messages=True,
)


def add_message(input, output):
    memory.save_context({"input": input}, {"output": output})


def get_history():
    return memory.load_memory_variables({})


add_message("Hi I'm Nicolas, I live in South Korea", "Wow that is so cool!")

add_message("South Korea is so pretty", "I wish I could go!!!")

add_message("How far is Brazil from Argentina?", "I don't know! Super far!")

add_message("How far is Brazil from Argentina?", "I don't know! Super far!")

In [ ]:
from langchain.memory import ConversationKGMemory
from langchain.chat_models import ChatOpenAI

llm = ChatOpenAI(temperature=0.1)

memory = ConversationKGMemory(
    llm=llm,
    return_messages=True,
)


def add_message(input, output):
    memory.save_context({"input": input}, {"output": output})


add_message("Hi I'm Nicolas, I live in South Korea", "Wow that is so cool!")

memory.load_memory_variables({"input": "who is Nicolas"})


add_message("Nicolas likes kimchi", "Wow that is so cool!")

memory.load_memory_variables({"inputs": "what does nicolas like"})

In [2]:
from langchain.chat_models import ChatOllama
from langchain.prompts import PromptTemplate, ChatPromptTemplate
from langchain.callbacks import StreamingStdOutCallbackHandler
from langchain.memory import ConversationSummaryBufferMemory
from langchain.chains import LLMChain

chat = ChatOllama(model="mistral:latest", streaming=True, callbacks=[StreamingStdOutCallbackHandler()])

memory = ConversationSummaryBufferMemory(
    llm=chat,
    max_token_limit=120,
    memory_key="chat_history",
)

template = """
    You are a helpful AI talking to a human.

    {chat_history}
    Human:{question}
    You:
"""

chain = LLMChain(
    llm=chat,
    memory=memory,
    prompt=PromptTemplate.from_template(template),
    verbose=True,
)

chain.predict(question = "My name is Nico")




NameError: name 'llm' is not defined

In [ ]:
chain.predict(question = "I live in tokyo")

In [ ]:
chain.predict(question = "What's my name?")